<a href="https://colab.research.google.com/github/srrmtt/AML/blob/main/step3_FedDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


!cp '/content/drive/MyDrive/client.py' './client.py'
!cp '/content/drive/MyDrive/server.py' './server.py'
!cp '/content/drive/MyDrive/dataset.py' './dataset.py'
!cp '/content/drive/MyDrive/bisenetv2.py' './bisenetv2.py'
!cp '/content/drive/MyDrive/transform.py' './transform.py'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torchvision
from sklearn.metrics import confusion_matrix  
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import VisionDataset
from torch.utils.data import Subset, DataLoader
from sklearn.model_selection import train_test_split
from PIL import Image
import os
import os.path
import sys
from torch.backends import cudnn
import numpy as np
import matplotlib.pyplot as plt

from dataset import Cityscapes
from client import *
from server import *
from bisenetv2 import *
from transform import *

In [3]:
#DEVICE = 'cuda' # 'cuda' or 'cpu'
DEVICE = 'cuda'
NUM_CLASSES = 19 # 101 + 1: There is am extra Background class that should be removed 

BATCH_SIZE = 16     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

LR = 5*1e-3           # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default -5

NUM_EPOCHS = 25      # 20/30 Total number of training epochs (iterations over dataset)
STEP_SIZE = 20       #20 How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10

In [4]:
#transformations
size = (256, 512) #512,1024
scales=(0.25, 2.)
cropsize=(512, 1024)
eval_crop=(1024, 1024)
eval_scales=(0.5, 0.75, 1.0, 1.25, 1.5, 1.75)
test_transformations = [RandomResizedCrop(eval_scales, eval_crop)]
train_transformations = [RandomResizedCrop(scales,cropsize),RandomHorizontalFlip(),ColorJitter(brightness=0.4,contrast=0.4,saturation=0.4)]
test_transformations = Compose(test_transformations)
train_transformations = Compose(train_transformations)

In [5]:

clients = []
datasets = []
with open("/content/drive/MyDrive/data/data/Cityscapes/train_A_I.txt") as lines:
  curr_id = 0
  paths = []
  for line in lines:
    id_client,path = line.split(" ")
    if curr_id == int(id_client):
      paths.append(path)
    else:
      print(f"finished Client ID:[{curr_id}]: len: {len(paths)}")
      dataset = DatasetClient("/content/drive/MyDrive/data/data/Cityscapes",paths,train_transformations)
      datasets.append(dataset)
      client = Client(curr_id,dataset)
      clients.append(client)

      paths = []
      paths.append(path)
      curr_id = int(id_client)  

  dataset = DatasetClient("/content/drive/MyDrive/data/data/Cityscapes",paths,train_transformations)
  datasets.append(dataset)
  client = Client(curr_id,dataset)
  clients.append(client)

print(len(clients), f"last: {curr_id}")

finished Client ID:[0]: len: 20
finished Client ID:[1]: len: 20
finished Client ID:[2]: len: 20
finished Client ID:[3]: len: 20
finished Client ID:[4]: len: 20
finished Client ID:[5]: len: 20
finished Client ID:[6]: len: 20
finished Client ID:[7]: len: 20
finished Client ID:[8]: len: 20
finished Client ID:[9]: len: 20
finished Client ID:[10]: len: 20
finished Client ID:[11]: len: 20
finished Client ID:[12]: len: 20
finished Client ID:[13]: len: 20
finished Client ID:[14]: len: 20
finished Client ID:[15]: len: 20
finished Client ID:[16]: len: 20
finished Client ID:[17]: len: 20
finished Client ID:[18]: len: 20
finished Client ID:[19]: len: 20
finished Client ID:[20]: len: 20
finished Client ID:[21]: len: 20
finished Client ID:[22]: len: 20
finished Client ID:[23]: len: 20
finished Client ID:[24]: len: 20
finished Client ID:[25]: len: 20
finished Client ID:[26]: len: 20
finished Client ID:[27]: len: 20
finished Client ID:[28]: len: 20
finished Client ID:[29]: len: 20
finished Client ID:[

In [6]:
#test_dataloader_B = DataLoader(test_dataset_B, batch_size=1, shuffle=False, num_workers=2)
#test_dataloader_A = DataLoader(test_dataset_A, batch_size=1, shuffle=False, num_workers=2)

In [7]:
model_path = "/content/drive/MyDrive/Results/Step2/LR5e-4_25epochs_16batchsize/dataset_A_test_mIoU/bisenet_best_mIoU_0.0005_tranformation (1).pt"
server = Server(clients, model_path)

server.train()



[Server]: round 0


AttributeError: ignored

In [ ]:
bisenet_model = BiSeNetV2(n_classes=NUM_CLASSES,output_aux=True,pretrained=False) #meglio toglierla perchè genera bias
bisenet_model.requires_grad = True

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=255,reduction='none') # da consegna ignore_index=255
parameters_to_optimize = bisenet_model.parameters() 
optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

In [ ]:
def compute_mIoU(y_true, y_pred,n_classes):
     # ytrue, ypred is a flatten vector
     y_pred = y_pred.cpu().flatten()
     y_true = y_true.cpu().flatten()
     current = confusion_matrix(y_true, y_pred, labels=[0, 1])
     # compute mean iou
     intersection = np.diag(current)
     ground_truth_set = current.sum(axis=1)
     predicted_set = current.sum(axis=0)
     union = ground_truth_set + predicted_set - intersection
     IoU = intersection / union.astype(np.float32)
     return np.mean(IoU)  

def compute_IoU(y_true, y_pred,n_classes):
     # ytrue, ypred is a flatten vector
     y_pred = y_pred.cpu().flatten()
     y_true = y_true.cpu().flatten()
     current = confusion_matrix(y_true, y_pred, labels=[0, 1])
     # compute mean iou
     intersection = np.diag(current)
     ground_truth_set = current.sum(axis=1)
     predicted_set = current.sum(axis=0)
     union = ground_truth_set + predicted_set - intersection
     IoU = intersection / union.astype(np.float32)
     return IoU 

def plotLoss(train):
  epochs = np.array([a for a in range(NUM_EPOCHS)]).reshape(NUM_EPOCHS,1)
  plt.figure()
  #plt.plot(epochs,val,label='val_loss')
  plt.plot(epochs,train,label='train_loss')
  plt.legend()
  plt.show()

def plotmIoU(train):
  epochs = np.array([a for a in range(NUM_EPOCHS)]).reshape(NUM_EPOCHS,1)
  plt.figure()
  #plt.plot(epochs,val,label='val_loss')
  plt.plot(epochs,train,label='train_mIoU')
  plt.legend()
  plt.show()

In [ ]:
net = bisenet_model.half().to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

cudnn.benchmark # Calling this optimizes runtime

#current_step = 0
# Start iterating over the epochs
#!nvidia-smi

loss_for_epochs = []
mIoU_for_epochs = []
best_mIoU = 0
file_path = "./bisenet_best_mIoU_"+ str(LR) + "_tranformation" + ".pt"
drive_path = "/content/drive/MyDrive/best_modelB_mIoU"+str(best_mIoU)+".pt"
for epoch in range(NUM_EPOCHS):
  print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))
  current_step=0
  # Iterate over the dataset
  for images, labels in train_dataloader_A:
    # Bring data over the device of choice
    
    images = images.half().to(DEVICE)
    labels = labels.half().to(DEVICE)
    #print("GPU Allocation after moving images and labels")
    #!nvidia-smi
    net.train() # Sets module in training mode

    # PyTorch, by default, accumulates gradients after each backward pass
    # We need to manually set the gradients to zero before starting a new iteration
    optimizer.zero_grad() # Zero-ing the gradients

    # Forward pass to the network
    output1,output2,output3,output4,output5 = net(images)
    pred1 = output1.argmax(dim=1)
    pred2 = output2.argmax(dim=1)
    pred3 = output3.argmax(dim=1)
    pred4 = output4.argmax(dim=1)
    pred5 = output5.argmax(dim=1)
    
    # #print(f"{pred1.size()}")
    # # print(type(images[0][0][0][0]))
    # # print("pred_size="+str(pred1.size()))
    # pred1_ = torch.flatten(pred1)
    # pred2_ = torch.flatten(pred2)
    # pred3_ = torch.flatten(pred3)
    # pred4_ = torch.flatten(pred4)
    # pred5_ = torch.flatten(pred5)


    # labels_ = torch.flatten(labels)
    loss1 = criterion(output1,labels.long())[labels!=255].mean()
    loss2 = criterion(output2,labels.long())[labels!=255].mean()
    loss3 = criterion(output3,labels.long())[labels!=255].mean()
    loss4 = criterion(output4,labels.long())[labels!=255].mean()
    loss5 = criterion(output5,labels.long())[labels!=255].mean()

    loss = loss1+loss2+loss3+loss4+loss5
    
    # Compute loss based on output and ground truth
    print("loss size:",loss.size())
    # Log loss
    if current_step % LOG_FREQUENCY == 0:
      print('Step {}, Loss {}'.format(current_step, loss.item()))
      print('pred1->{} labels->{}'.format(pred1.size(),labels.size()))
      #inserire unsqueeze alla posizione 1
      #pred1 = torch.unsqueeze(pred1,1)
      #labels = torch.unsqueeze(labels,1)
      #print('pred1->{} labels->{}'.format(pred1.size(),labels.size()))
      mIoU = compute_mIoU(labels,pred1,n_classes=19)
      if best_mIoU < mIoU:
        print("NEW BEST mIoU!")
        best_mIoU = mIoU
        torch.save(bisenet_model.state_dict(), file_path)
      print('Step {}, mIoU {}'.format(current_step, mIoU))

    # Compute gradients for each layer and update weights
    #loss1.requires_grad=True
    #loss2.requires_grad=True
    #loss3.requires_grad=True
    #loss4.requires_grad=True
    #loss5.requires_grad=True

    #loss.requires_grad = True
    loss.backward()
    
    #loss1.backward()  # backward pass: computes gradients
    #loss2.backward()  # backward pass: computes gradients
    #loss3.backward()  # backward pass: computes gradients
    #loss4.backward()  # backward pass: computes gradients
    #loss5.backward()  # backward pass: computes gradients

    #torch.cuda.empty_cache()    

  
    optimizer.step() # update weights based on accumulated gradients

    current_step += 1

  loss_for_epochs.append(loss.item())
  mIoU_for_epochs.append(mIoU)
  # Step the scheduler
  scheduler.step() 

plotLoss(loss_for_epochs)
plotmIoU(mIoU_for_epochs)
!cp file_path drive_path

In [ ]:
#torch.save(bisenet_model.state_dict(), "/content/drive/MyDrive/Results/Step2/LR5e-1_10epochs_batchsize16/bisenet_best_mIoU3e-1_0.5_tranformation.pt")
#net = BiSeNetV2(n_classes=20)
#net.load_state_dict(torch.load("/content/drive/MyDrive/Results/Step2/LR1e-0_10epochs_batchsize16/datasetB_mIoU_1.0_tranformation/bisenet_best_mIoU_1.0_tranformation (1).pt"),strict=False)

In [ ]:
from torchvision import ops
from tqdm import tqdm
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
net.train(False) # Set Network to evaluation mode
net=net.half()
running_corrects = 0
torch.cuda.empty_cache() 
mIoU = 0
count = 0
for images, labels in tqdm(test_dataloader_A):
  images = images.half().to(DEVICE)
  labels = labels.half().to(DEVICE)
  # print(images.size())
  # print(labels.size())
  # Forward Pass
  outputs = net(images,test=True,use_test_resize=False)
  preds = outputs.argmax(dim=1)
  # print(outputs.size())
  mIoU += compute_mIoU(labels,preds,NUM_CLASSES)
  count += 1
  
  # iou = ops.box_iou(labels, outputs)

  # print('IOU : ', iou.numpy()[0][0])
print("mIoU = ",mIoU/count)